In [1]:
import pandas as pd
import numpy as np

file_path='./Assessor_Historical_Secured_Property_Tax_Rolls.csv'
data=pd.read_csv(file_path,low_memory=False)
data.head

<bound method NDFrame.head of          Closed Roll Year                     Property Location Parcel Number  \
0                  2007.0  0000 0000                       0000       1378007   
1                  2008.0  0000 0000                       0000       1345001   
2                  2008.0  0000 0000                       0000       5607046   
3                  2009.0  0000 0297 BALTIMORE           WY0000       6499040   
4                  2010.0  0000 1715 SITUS TO BE ASSIGNEDST0000       0663054   
...                   ...                                   ...           ...   
2029420            2016.0  0000 0000 THE EMBARCADERO       0000       9900275   
2029421            2016.0  0000 0000 THE EMBARCADERO       0000       9900278   
2029422            2016.0  0000 0000 BERRY               ST0000       9900336   
2029423            2016.0  0000 0000 THE EMBARCADERO       0039       9900501   
2029424            2016.0  0000 0000 THE EMBARCADERO       0039       9900502  

In [2]:
ActualFrame=data[['Closed Roll Year', 'Property Area', 'Assessed Land Value']]
ActualFrame=ActualFrame.dropna()

ActualFrame = ActualFrame.loc[ActualFrame['Property Area']>0]
ActualFrame["Norm Value/ft^2"]=ActualFrame['Assessed Land Value']/ActualFrame['Property Area']
ActualFrame.index=ActualFrame['Closed Roll Year']
ActualFrame
AveragedValues=ActualFrame.groupby(level=0).mean()
AveragedValues
# Sum the 4 value columns and then do a regression of the sum of the assessed vales against the current sales date
# Use this linear regression to predict the cost of the current home in the future


,Closed Roll Year,Property Area,Assessed Land Value,Norm Value/ft^2
Closed Roll Year,,,,
2007.0,2007.0,3242.986355,296978.883627,141.577561
2008.0,2008.0,3218.368840,324391.820970,153.602812
2009.0,2009.0,3228.545849,350271.567000,166.528898
2010.0,2010.0,3218.356434,365860.964753,169.301822
2011.0,2011.0,3208.832037,378936.998891,170.363308
2012.0,2012.0,3150.776963,392661.929355,178.774641
2013.0,2013.0,3162.832667,411945.776749,221.301145
2014.0,2014.0,3169.076873,433292.047195,232.758056
2016.0,2016.0,3177.380458,490514.526687,461.222437


In [3]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import matplotlib.pyplot as plt
X=np.array(AveragedValues['Closed Roll Year'])
Y=np.array(AveragedValues['Norm Value/ft^2'])
x_train, x_test, y_train, y_test=train_test_split(X,Y, test_size=0.2, random_state=100)
print(x_train)
x_train= np.reshape(x_train,(-1, 1))
y_train= np.reshape(y_train,(-1, 1))
x_test = np.reshape(x_test, (-1, 1))
y_test = np.reshape(y_test, (-1, 1))
X= np.reshape(X,(-1,1))
Y= np.reshape(Y,(-1,1))
LinearValueModel=linear_model.LinearRegression()
LinearValueModel.fit(X,Y)
print(LinearValueModel.score(X,Y))
plt.plot(X,Y)
plt.show()

[2008. 2012. 2011. 2009. 2007. 2010. 2017. 2016.]
0.7437914722673638


<Figure size 640x480 with 1 Axes>

In [4]:
import pickle
filename='value_model.sav'
pickle.dump(LinearValueModel, open(filename, 'wb'))